In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.max_rows', 500)

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('data/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' column, count(*) - count(CRIM) n_empty FROM boston
    UNION ALL
    SELECT 'ZN', count(*) - count(ZN) FROM boston
    UNION ALL
    SELECT 'INDUS', count(*) - count(INDUS) FROM boston
    UNION ALL
    SELECT 'CHAS', count(*) - count(CHAS) FROM boston
    UNION ALL
    SELECT 'NOX', count(*) - count(NOX) FROM boston
         
    ''',
    con,
)

,column,n_empty
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' column, count(distinct CRIM) n_unique FROM boston
    UNION ALL
    SELECT 'ZN', count(distinct ZN) FROM boston
    UNION ALL
    SELECT 'INDUS', count(distinct INDUS) FROM boston
    UNION ALL
    SELECT 'CHAS', count(distinct CHAS) FROM boston
    UNION ALL
    SELECT 'NOX', count(distinct NOX) FROM boston
    ''',
    con,
)

,column,n_unique
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks+=1
pd.read_sql(
    '''
    WITH cte AS (
        SELECT CRIM, ZN, INDUS, CHAS, NOX, 
            row_number() over (order by CRIM) crim_rn, 
            row_number() over (order by ZN) zn_rn, 
            row_number() over (order by INDUS) indus_rn, 
            row_number() over (order by CHAS) chas_rn, 
            row_number() over (order by NOX) nox_rn, 
            count(*) over () cnt
        FROM boston
    )
    SELECT column FROM (
        SELECT 'ZN' column, MIN(ZN) min, AVG(CASE WHEN zn_rn IN (cnt / 2 + 1, (cnt - 1 + cnt % 2) / 2 + 1) THEN ZN END) median FROM cte
        UNION ALL
        SELECT 'CRIM', MIN(CRIM), AVG(CASE WHEN crim_rn IN (cnt / 2 + 1, (cnt - 1 + cnt % 2) / 2 + 1) THEN CRIM END) FROM cte
        UNION ALL
        SELECT 'INDUS', MIN(INDUS), AVG(CASE WHEN indus_rn IN (cnt / 2 + 1, (cnt - 1 + cnt % 2) / 2 + 1) THEN INDUS END) FROM cte
        UNION ALL
        SELECT 'CHAS', MIN(CHAS), AVG(CASE WHEN chas_rn IN (cnt / 2 + 1, (cnt - 1 + cnt % 2) / 2 + 1) THEN CHAS END) FROM cte
        UNION ALL
        SELECT 'NOX', MIN(NOX), AVG(CASE WHEN nox_rn IN (cnt / 2 + 1, (cnt - 1 + cnt % 2) / 2 + 1) THEN NOX END) FROM cte
    ) WHERE min = median
    ''',
    con,
)
# Выводы:
# ZN: более половины районов Бостона имеют минимальное кол-во участков площадью от 25000 кв. футов 
#     (точнее, не имеют их вообще, т.к. `data.ZN.min() == 0.0`)
# CHAS: более половины районов Бостона не имеют доступа к реке
#

,column
0,ZN
1,CHAS


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql(
    '''
    WITH cheap AS (
        SELECT g.grp, AVG(b.RM) cheap_rm FROM (
            SELECT 25 grp UNION ALL SELECT 50 UNION ALL SELECT 100 UNION ALL SELECT 200 UNION ALL SELECT 300) g
        JOIN (
            SELECT RM, row_number() over (order by MEDV, RM) rn from boston) b
        ON b.rn <= g.grp
        GROUP BY grp
    ),
    luxury AS (
        SELECT AVG(RM) luxury_rm FROM boston WHERE MEDV = (SELECT max(MEDV) FROM boston)
    )
    SELECT grp, cheap_rm, luxury_rm, luxury_rm - cheap_rm diff FROM cheap CROSS JOIN luxury
    ''',
    con,
)
# Замечания: 
#   * дополнительную сортировку по RM сделал для воспроизводимости результатов в pandas
#     (присутствуют разные значения RM для одного и того же MEDV)
#   * формулировка задания не вполне корректна, т.к. датасет содержит агрегированную информацию о всех домах 
#     в каждом районе Бостона, а не о конкретном доме. Так что лучше говорить о сравнении дорогих и дешевых районов
#     по средней (медианной) цене дома
#
# Выводы:
#   * среднее количество комнат в районах с дорогими домами выше чем в бедных районах
#   * оценить насколько сильно количество комнат влияет на стоимость достаточно сложно по данной выборке, 
#     т.к. на стоимость влияют и другие факторы (расположение, социальный статус жителей района и т.д.), 
#     но можно заметить что с ростом числа бедных районов в выборке разница в количестве комнат уменьшается незначительно

,grp,cheap_rm,luxury_rm,diff
0,25,5.69412,7.484,1.78988
1,50,5.75324,7.484,1.73076
2,100,5.87511,7.484,1.60889
3,200,5.91122,7.484,1.57278
4,300,5.97121,7.484,1.51279


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM (
        SELECT LSTAT, MEDV, rank() over (ORDER BY LSTAT) rnk FROM boston
    )
    WHERE MEDV = (SELECT max(MEDV) FROM boston)
    ''',
    con,
)

# Замечание:
#   * насколько я понимаю, LSTAT - это количество домохозяйств в районе 
#     с низким социально-экономичесаим статусом (проще говоря, за чертой бедности)
#   * чем выше ранг, тем больше бедняков в районе
#
# Выводы:
#   * видно, что в самых дорогих районах (по цене за дом) доля бедняков очень мала (низкий LSTAT)
#   * хотя внизу списка есть районы, где доля бедняков повышена
#     (LSTAT для последней строки в 5.5 раз выше чем для первой, а также высокий ранг)

,LSTAT,MEDV,rnk
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT CASE WHEN CHAS = 0 THEN 'не граничит' ELSE 'граничит' END river, AVG(MEDV) avg_price 
    FROM boston GROUP BY CHAS
    ''',
    con,
)

# Вывод:
#   * в районах, граничащих с рекой, стоимость домов выше

,river,avg_price
0,не граничит,22.093843
1,граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT column FROM (
        SELECT 'CRIM' column, AVG(CASE WHEN CHAS=0 THEN CRIM END) no_river, AVG(CASE WHEN CHAS=1 THEN CRIM END) has_river FROM boston
        UNION ALL
        SELECT 'ZN', AVG(CASE WHEN CHAS=0 THEN ZN END), AVG(CASE WHEN CHAS=1 THEN ZN END) FROM boston
        UNION ALL
        SELECT 'INDUS', AVG(CASE WHEN CHAS=0 THEN INDUS END), AVG(CASE WHEN CHAS=1 THEN INDUS END) FROM boston
        UNION ALL
        SELECT 'CHAS', AVG(CASE WHEN CHAS=0 THEN CHAS END), AVG(CASE WHEN CHAS=1 THEN CHAS END) FROM boston
        UNION ALL
        SELECT 'NOX', AVG(CASE WHEN CHAS=0 THEN NOX END), AVG(CASE WHEN CHAS=1 THEN NOX END) FROM boston
    )
    WHERE has_river > no_river
    ''',
    con,
)

# Выводы:
#   * INDUS: если правильно понял смысл этой колонки (доля промзон), то доля промзон выше в районах у реки 
#     (видимо, чтобы сбрасывать туда отходы)
#   * CHAS: вывод тривиальный (мы по этой колонке и делаем разбиение) - в районах у реки есть река
#   * NOX: концентрация оксидов азота выше в районах у реки 
#     (возможно есть значительная положительная корреляция с долей промзон в районе (INDUS))
#

,column
0,INDUS
1,CHAS
2,NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
tasks+=1
pd.read_sql(
    '''
    WITH indus_pct AS (
        SELECT pct, max(INDUS) indus_val FROM (
            SELECT INDUS, (ntile(10) over (ORDER BY INDUS)) * 10 pct FROM boston
        ) GROUP BY pct
    ),
    nox_pct AS(
        SELECT pct, max(NOX) nox_val FROM (
            SELECT NOX, (ntile(10) over (ORDER BY NOX)) * 10 pct FROM boston
        ) GROUP BY pct
    )
    SELECT i.pct, indus_val, nox_val FROM indus_pct i JOIN nox_pct n ON i.pct = n.pct
    ''',
    con,
)

# Вывод: по данной выборке видно, что значения растут с разной скоростью: для INDUS разница max/min ~ 9, а для NOX ~ 2.
# Явной взаимосвязи не прослеживается

,pct,indus_val,nox_val
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,80,18.10,0.671
8,90,19.58,0.713
9,100,27.74,0.871


In [15]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
